In [0]:
from pyspark.sql.window import Window

In [0]:
catalog_name = "retail_dev"
schema_bronze = "bronze"
schema_silver = "silver"
schema_gold = "gold"
schema_auditoria ="auditoria"

**Ticket promedio por tienda y mes**


In [0]:

# 1. Cargar las tablas necesarias como DataFrames
fact_ventas = spark.table(f"{catalog_name}.{schema_gold}.fact_ventas")
dim_tienda = spark.table(f"{catalog_name}.{schema_gold}.dim_tiendas")
dim_tiempo = spark.table(f"{catalog_name}.{schema_gold}.dim_tiempo")

# 2. Unir (join) los DataFrames
kpi_df = (
    fact_ventas.alias("fv")
    .join(dim_tienda.alias("ti"), "id_tienda")
    .join(dim_tiempo.alias("tiem"), "id_tiempo")
)

# 3. Agrupar y calcular el KPI
ticket_promedio_df = (
     kpi_df.groupBy("ti.nombre", "tiem.anio", "tiem.mes")
    .agg(
        F.when(
            F.sum("fv.cantidad") > 0,
            F.sum("fv.monto") / F.sum("fv.cantidad")
        ).otherwise(0).alias("ticket_promedio")
    )
    .withColumnRenamed("nombre", "tienda")
    .orderBy(F.col("anio").desc(), F.col("mes").desc(), F.col("tienda"))
)

# 4. Mostrar el resultado
display(ticket_promedio_df)

In [0]:

ticket_promedio_df.createOrReplaceTempView("v_kpi_ticket_promedio_tienda_mes_py")

**Clientes nuevos por mes**

In [0]:
# 1. Cargar la tabla de hechos
fact_ventas = spark.table(f"{catalog_name}.{schema_gold}.fact_ventas")

# 2. Encontrar la primera fecha de compra para cada cliente
primera_compra_df = (
    fact_ventas
    .groupBy("id_cliente")
    .agg(F.min("fecha_venta").alias("fecha_primera_compra"))
)

# 3. Agrupar por mes y contar los clientes nuevos
clientes_nuevos_mes_df = (
    primera_compra_df
    # Extraer el 'año-mes' de la fecha de primera compra
    .withColumn("anio_registro", F.year("fecha_primera_compra"))
    .withColumn("mes_registro", F.month("fecha_primera_compra"))
    .groupBy("anio_registro", "mes_registro")
    .count()
    .withColumnRenamed("count", "cantidad_clientes_nuevos")
    .orderBy(F.col("anio_registro").desc(), F.col("mes_registro").desc())
)



In [0]:
display(clientes_nuevos_mes_df)

In [0]:
clientes_nuevos_mes_df.createOrReplaceTempView("v_kpi_clientes_nuevos_mes_py")

**Código PySpark para Tasa de Devolución Mensual**

In [0]:
# 1. Cargar las tablas necesarias
fact_ventas = spark.table(f"{catalog_name}.{schema_gold}.fact_ventas")
dim_tiempo = spark.table(f"{catalog_name}.{schema_gold}.dim_tiempo")
devoluciones = spark.table(f"{catalog_name}.{schema_silver}.devoluciones")

# 2. Calcular el total de ventas únicas por mes
ventas_mensuales = (
    fact_ventas.alias("fv")
    .join(dim_tiempo.alias("dt"), "id_tiempo")
    # Agrupamos por año y mes para obtener el periodo
    .groupBy("dt.anio", "dt.mes")
    # Contamos el número de ventas distintas (no líneas de venta)
    .agg(F.countDistinct("fv.id_venta").alias("total_ventas"))
)

# 3. Calcular el total de devoluciones por mes
devoluciones_mensuales = (
    devoluciones.alias("d")
    # Se une con dim_tiempo usando la fecha de la devolución
    .join(dim_tiempo.alias("dt"), F.to_date(F.col("d.fecha_devolucion")) == F.col("dt.fecha"))
    .groupBy("dt.anio", "dt.mes")
    # Contamos el número de devoluciones
    .agg(F.count("d.id_devolucion").alias("total_devoluciones"))
)

# 4. Unir los resultados y calcular la tasa de devolución
tasa_devolucion_df = (
    ventas_mensuales
    # Usamos un 'full_outer' join para no perder meses que tengan ventas pero no devoluciones (o viceversa)
    .join(devoluciones_mensuales, ["anio", "mes"], "full_outer")
    # Rellenamos con 0 los nulos que puedan resultar del join
    .na.fill(0)
    .withColumn(
        "tasa_devolucion",
        # Fórmula: (# devoluciones / # ventas)
        # Se usa F.when para evitar la división por cero
        F.when(
            F.col("total_ventas") > 0,
            F.col("total_devoluciones") / F.col("total_ventas")
        ).otherwise(0) # Si no hubo ventas, la tasa es 0
    )
    .orderBy(F.col("anio").desc(), F.col("mes").desc())
)


In [0]:
display(tasa_devolucion_df)

In [0]:
tasa_devolucion_df.createOrReplaceTempView("v_kpi_tasa_devolucion_py")

**Crecimiento MoM por categoría**

In [0]:
# 1. Cargar las tablas necesarias
fact_ventas = spark.table(f"{catalog_name}.{schema_gold}.fact_ventas")
dim_producto = spark.table(f"{catalog_name}.{schema_gold}.dim_productos")
dim_tiempo = spark.table(f"{catalog_name}.{schema_gold}.dim_tiempo")

# 2. Calcular las ventas totales por categoría y mes
ventas_mensuales_categoria = (
    fact_ventas.alias("fv")
    .join(dim_producto.alias("dp"), "id_producto")
    .join(dim_tiempo.alias("dt"), "id_tiempo")
    .groupBy("dp.categoria", "dt.anio", "dt.mes")
    .agg(F.sum("fv.monto").alias("ventas_mes_actual"))
)

# 3. Definir la función de ventana para obtener las ventas del mes anterior
# La ventana se divide por 'categoria' y se ordena por fecha (año y mes)
window_spec = Window.partitionBy("categoria").orderBy("anio", "mes")

# 4. Aplicar la función LAG para traer el dato del mes anterior a la fila actual
ventas_con_mes_anterior = (
    ventas_mensuales_categoria
    .withColumn(
        "ventas_mes_anterior",
        F.lag("ventas_mes_actual").over(window_spec)
    )
)

# 5. Calcular la tasa de crecimiento MoM
crecimiento_mom_df = (
    ventas_con_mes_anterior
    .withColumn(
        "crecimiento_mom",
        # Fórmula: (actual - anterior) / anterior
        # Se usa F.when para manejar el primer mes (donde el anterior es nulo) y evitar división por cero
        F.when(
            (F.col("ventas_mes_anterior").isNotNull()) & (F.col("ventas_mes_anterior") != 0),
            (F.col("ventas_mes_actual") - F.col("ventas_mes_anterior")) / F.col("ventas_mes_anterior")
        ).otherwise(None) # El primer mes no tiene crecimiento, se deja como nulo
    )
    .orderBy(F.col("categoria"), F.col("anio"), F.col("mes"))
)



In [0]:
display(crecimiento_mom_df)

In [0]:
crecimiento_mom_df.createOrReplaceTempView("v_kpi_crecimiento_mom_df_py")

**Resumen incremental de la última corrida**

In [0]:
# 1. Cargar la tabla de auditoría
log_df = spark.table(f"{catalog_name}.{schema_auditoria}.ingestion_log")

# 2. Obtener el registro de la última corrida
# Se ordena por la fecha de creación en orden descendente y se toma el primer registro
ultima_corrida_df = (
    log_df
    .orderBy(F.col("creation_date").desc())
    .limit(1)
    .select(
        "creation_date",
        "table",
        "rows_inserted",
        "rows_updated",
        "rows_deleted",
        "merge_duration_seconds"
    )
)


In [0]:

display(ultima_corrida_df)

In [0]:
ultima_corrida_df.createOrReplaceTempView("v_kpi_ultima_corrida_df_py")